In [ ]:
%pip -q install ultralytics opencv-python pyyaml onvif-zeep

## Fun with cameras and AI

In [ ]:
import os, requests, base64
from pprint import pprint
import cv2
from onvif import ONVIFCamera

def get_rtsp_url_from_onvif(host, port, user, passwd):
    camera = ONVIFCamera(host, port, user, passwd)
    media_service = camera.create_media_service()
    profiles = media_service.GetProfiles()
    token = profiles[0].token
    stream_uri = media_service.GetStreamUri({'StreamSetup': {'Stream': 'RTP-Unicast', 'Transport': 'RTSP'}, 'ProfileToken': token})
    return stream_uri.Uri

def view_rtsp_feed(rtsp_url):
    cap = cv2.VideoCapture(rtsp_url)
    
    if not cap.isOpened():
        print("Error: Could not open RTSP stream.")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to retrieve frame.")
            break
        
        cv2.imshow('RTSP Feed', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Call the AI service API
def call_api(uri, key, payload):
    headers = {
        "Content-Type": "application/json",
        "api-key": key,
    }
    # Send request
    try:
        response = requests.post(uri, headers=headers, json=payload)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    except requests.RequestException as e:
        raise SystemExit(f"Failed to make the request. Error: {e}")

    # Handle the response as needed (e.g., print or process)
    response_json = response.json()
    return response_json

# Convert an image to base64 encoding
def image_to_base64(image_path_or_url):
    if image_path_or_url.startswith('http://') or image_path_or_url.startswith('https://'):
        response = requests.get(image_path_or_url)
        image_data = response.content
    else:
        with open(image_path_or_url, "rb") as image_file:
            image_data = image_file.read()
    return base64.b64encode(image_data).decode('utf-8')



In [ ]:
# Example usage
host = "192.168.2.245"
port = 80
user = "admin"
passwd = ""

In [ ]:
rtsp_url = get_rtsp_url_from_onvif(host, port, user, passwd)
view_rtsp_feed(rtsp_url)